# Всероссийская олимпиада по искусственному интеллекту 2025

## Введение в задачу Keyword Spotting

Добро пожаловать в задачу **Не слышу тебя**, которая является примером **Keyword Spotting (KWS)** — распознавания ключевых фраз в аудиозаписях. В этом ноутбуке показано, как подготовить данные к работе с ними.

### Описание задачи

Во время телефонных или видеозвонков качество связи может резко ухудшаться: звук пропадает, речь прерывается, собеседники перестают друг друга слышать. Один из способов автоматического обнаружения таких проблем — анализ аудиопотока на предмет ключевых фраз, по которым можно определить, что связь прервалась.

В этой задаче рассматриваются две такие фразы: «не слышу» и «не слышно». Их появление в разговоре сигнализирует о том, что пользователь испытывает проблемы с восприятием речи собеседника, что может быть связано с плохим интернет-соединением, неисправностью аудиотракта или помехами в канале связи.

Создание и внедрение такой модели позволит:
- оперативно определять ухудшение качества связи в реальном времени;
- автоматизировать сбор статистики о проблемах в звонках;
- инициировать технические меры для улучшения соединения (например, переключение кодека или перезапуск соединения);
- использовать полученную информацию для аналитики и повышения качества сервиса.

### Цель задачи

Необходимо построить модель, которая для каждой аудиозаписи определит, **содержит ли она хотя бы одну из целевых фраз «не слышу» или «не слышно»**.

- Метка **1** — в записи есть хотя бы одна из этих фраз.
- Метка **0** — таких фраз нет.

Таким образом, задача представляет собой **бинарную классификацию аудиозаписей**: модель должна различать записи с целевыми словами и без них.

**New model wavWec**

In [12]:
import math
import random
from pathlib import Path
from typing import List, Tuple, Dict, Optional, Union
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import numpy as np
import librosa
from tqdm import tqdm
import json
from glob import glob
from sklearn.model_selection import train_test_split

# ДОБАВЛЕННЫЕ ИМПОРТЫ
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import CSVLogger
import torch.optim as optim
from transformers import AutoModel, AutoProcessor, Wav2Vec2Processor, Wav2Vec2Model
from torchmetrics import F1Score, Accuracy, Precision, Recall
import torch.nn as nn

In [13]:
SEED = 185

def set_all_seeds(seed: int = SEED):
    """
    Устанавливает сид для Python, NumPy, PyTorch (CPU и GPU) и PyTorch Lightning.
    """
    print(f"⚙️ Установка SEED: {seed}")
    
    # 1. Системные и Python
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    
    # 2. NumPy
    np.random.seed(seed)
    
    # 3. PyTorch (CPU и GPU)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        # Для детерминированного поведения GPU
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False # Может немного замедлить
        
    # 4. PyTorch Lightning
    pl.seed_everything(seed, workers=True)

In [ ]:
# КОНСТАНТЫ ДЛЯ НАСТРОЙКИ
class Config:
    # Аудио параметры
    SEED = SEED
    SAMPLE_RATE = 16000
    SEGMENT_DURATION = 2.0  # секунды
    SEGMENT_SIZE = int(SAMPLE_RATE * SEGMENT_DURATION)  # 16000 samples
    
    # Обучение
    BATCH_SIZE = 32  # Уменьшил из-за большой модели
    LEARNING_RATE = 1e-4
    MAX_EPOCHS = 3
    NUM_WORKERS = 8
    
    # Аугментации
    AUGMENT_PROB = 0.7
    NOISE_SNR_RANGE = (15, 30)
    TIME_SHIFT_MAX = 0.1
    GAIN_RANGE = (0.7, 1.3)
    TIME_STRETCH_RATE = 0.15
    PITCH_SHIFT_RANGE = (-1, 1)
    BACKGROUND_NOISE_RANGE = (0.01, 0.05)
    # model_name = "facebook/wav2vec2-base-960h"

    # # Используйте одну из этих русскоязычных моделей:
    # model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"  # Наиболее популярная
    # # ИЛИ
    # model_name = "bond005/wav2vec2-large-ru-golos"  # Обучалась на русских данных
    # # ИЛИ  
    model_name = "UrukHan/wav2vec2-russian"  # С дополнительной коррекцией текста
    
    CHECKPOINT_DIR = "./checkpoints"
    BEST_MODEL_PATH = "./best_kws_model.pth"

# Расширенные функции аугментации аудио
class AudioAugmentations:
    """Профессиональные аугментации для аудио"""
    
    @staticmethod
    def add_gaussian_noise(audio: torch.Tensor, snr_db: float = 20) -> torch.Tensor:
        """Добавление гауссовского шума с заданным SNR"""
        std = audio.std()
        if std < 1e-6:
            return audio
        noise_std = std / (10 ** (snr_db / 20))
        noise = torch.randn_like(audio) * noise_std
        return audio + noise
    
    @staticmethod
    def time_shift(audio: torch.Tensor, max_shift: float = 0.2) -> torch.Tensor:
        """Сдвиг во времени"""
        shift = int(random.uniform(-max_shift, max_shift) * len(audio))
        if shift > 0:
            shifted = torch.cat([audio[shift:], torch.zeros(shift, device=audio.device)])
        elif shift < 0:
            shifted = torch.cat([torch.zeros(-shift, device=audio.device), audio[:shift]])
        else:
            shifted = audio
        return shifted
    
    @staticmethod
    def pitch_shift(audio: torch.Tensor, sr: int, n_steps: float = 2) -> torch.Tensor:
        """Сдвиг тона (pitch shifting)"""
        try:
            audio_np = audio.cpu().numpy()
            shifted = librosa.effects.pitch_shift(audio_np, sr=sr, n_steps=n_steps)
            return torch.from_numpy(shifted).to(audio.device)
        except:
            return audio
    
    @staticmethod
    def time_stretch(audio: torch.Tensor, rate: float = 0.2) -> torch.Tensor:
        """Растяжение/сжатие времени"""
        try:
            min_rate = 1.0 - rate
            max_rate = 1.0 + rate
            stretch_rate = random.uniform(min_rate, max_rate)
            
            audio_np = audio.cpu().numpy()
            stretched = librosa.effects.time_stretch(audio_np, rate=stretch_rate)
            
            # Приведение к исходной длине
            if len(stretched) > len(audio):
                stretched = stretched[:len(audio)]
            else:
                stretched = np.pad(stretched, (0, max(0, len(audio) - len(stretched))))
                
            return torch.from_numpy(stretched).to(audio.device)
        except:
            return audio
    
    @staticmethod
    def random_gain(audio: torch.Tensor, min_gain: float = 0.5, max_gain: float = 1.5) -> torch.Tensor:
        """Случайное изменение громкости"""
        gain = random.uniform(min_gain, max_gain)
        return audio * gain
    
    @staticmethod
    def background_noise(audio: torch.Tensor, noise_level: float = 0.05) -> torch.Tensor:
        """Добавление фонового шума"""
        if audio.std() < 1e-6:
            return audio
        noise = torch.randn_like(audio) * noise_level * audio.std()
        return audio + noise

# Функции загрузки аудио
def load_audio_16k(path: Union[str, Path], sr_target: int = Config.SAMPLE_RATE) -> torch.Tensor:
    path = str(path)
    try:
        wav, sr = torchaudio.load(path)
        if wav.dim() == 2:
            if wav.size(0) > 1:
                wav = wav.mean(dim=0, keepdim=True)
            wav = wav.squeeze(0)

        if sr != sr_target:
            wav = torchaudio.functional.resample(wav, orig_freq=sr, new_freq=sr_target)

        if wav.dtype != torch.float32:
            wav = wav.float()
        return wav
    except Exception as e:
        try:
            y, sr = librosa.load(path, sr=sr_target, mono=True)
            wav = torch.from_numpy(y.astype(np.float32))
            return wav
        except Exception as e2:
            return torch.zeros(sr_target, dtype=torch.float32)

def ensure_length(wav: torch.Tensor, target_len: int) -> torch.Tensor:
    T = wav.numel()
    if T == target_len:
        return wav
    if T > target_len:
        return wav[:target_len]
    pad = target_len - T
    return F.pad(wav, (0, pad))

def pick_positive_window(T: int, sr: int, seg_size: int, bounds: Tuple[float, float]) -> Tuple[int, int]:
    t0, t1 = bounds
    p0 = int(round(t0 * sr))
    p1 = int(round(t1 * sr))
    p0 = max(0, min(T, p0))
    p1 = max(0, min(T, p1))
    pos_len = max(1, p1 - p0)

    if pos_len >= seg_size:
        center = (p0 + p1) // 2
        left = max(0, center - seg_size // 2)
        right = min(T, left + seg_size)
        left = right - seg_size
        return left, right

    free = seg_size - pos_len
    alpha = np.clip(np.random.normal(loc=0.5, scale=0.15), 0.0, 1.0)
    left_ctx = int(alpha * free)
    right_ctx = free - left_ctx

    left = p0 - left_ctx
    right = p1 + right_ctx

    if left < 0:
        shift = -left
        left = 0
        right = min(T, right + shift)
    if right > T:
        shift = right - T
        right = T
        left = max(0, left - shift)

    if right - left != seg_size:
        right = min(T, left + seg_size)
        left = right - seg_size
        left = max(0, left)
    return left, right

def pick_negative_window(T: int, seg_size: int) -> Tuple[int, int]:
    if T <= seg_size:
        return 0, min(T, seg_size)
    left = random.randint(0, T - seg_size)
    right = left + seg_size
    return left, right

In [15]:
# Улучшенный датасет с расширенными аугментациями
class AdvancedKWSDataset(Dataset):
    def __init__(self, file_list: List[Tuple[Path, Optional[Tuple[float, float]], int]], 
                 seg_size_samples: int = Config.SEGMENT_SIZE, 
                 sr: int = Config.SAMPLE_RATE, 
                 augment: bool = False):
        super().__init__()
        self.sr = sr
        self.seg_size = seg_size_samples
        self.augment = augment
        self.samples = file_list
        self.augmentor = AudioAugmentations()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index: int):
        path, bounds, label = self.samples[index]
        
        wav = load_audio_16k(path, sr_target=self.sr)
        peak = wav.abs().max().item()
        if peak > 1e-6:
            wav = wav / peak

        T = wav.numel()
        
        if T == 0:
            segment = torch.zeros(self.seg_size, dtype=torch.float32)
        elif label == 1 and bounds is not None:
            left, right = pick_positive_window(T=T, sr=self.sr, seg_size=self.seg_size, bounds=bounds)
            segment = wav[left:right]
            segment = ensure_length(segment, self.seg_size)
        else:
            left, right = pick_negative_window(T=T, seg_size=self.seg_size)
            segment = wav[left:right]
            segment = ensure_length(segment, self.seg_size)

        # Расширенные аугментации
        if self.augment:
            # Гауссовский шум
            if random.random() > Config.AUGMENT_PROB:
                snr_db = random.uniform(*Config.NOISE_SNR_RANGE)
                segment = self.augmentor.add_gaussian_noise(segment, snr_db=snr_db)
            
            # Сдвиг во времени
            if random.random() > Config.AUGMENT_PROB:
                segment = self.augmentor.time_shift(segment, max_shift=Config.TIME_SHIFT_MAX)
            
            # Изменение громкости
            if random.random() > 0.5:
                segment = self.augmentor.random_gain(segment, *Config.GAIN_RANGE)
            
            # Растяжение времени
            if random.random() > Config.AUGMENT_PROB:
                segment = self.augmentor.time_stretch(segment, rate=Config.TIME_STRETCH_RATE)
            
            # Сдвиг тона (только для положительных примеров)
            if label == 1 and random.random() > Config.AUGMENT_PROB:
                n_steps = random.uniform(*Config.PITCH_SHIFT_RANGE)
                segment = self.augmentor.pitch_shift(segment, self.sr, n_steps=n_steps)
            
            # Фоновый шум
            if random.random() > Config.AUGMENT_PROB:
                noise_level = random.uniform(*Config.BACKGROUND_NOISE_RANGE)
                segment = self.augmentor.background_noise(segment, noise_level=noise_level)

        aux = {"path": str(path), "label": label}
        return segment, label, aux

def custom_collate_fn(batch):
    segments, labels, auxs = zip(*batch)
    segments = torch.stack(segments)
    labels = torch.tensor(labels, dtype=torch.long)
    return segments, labels, auxs

# ЗАГРУЗКА ДАННЫХ
def load_and_prepare_data():
    """ШАГ 1: Загрузка и подготовка всех данных"""
    print("=== ЗАГРУЗКА ДАННЫХ ===")
    print(f"Конфигурация: сегмент {Config.SEGMENT_DURATION} сек ({Config.SEGMENT_SIZE} samples)")
    
    train_dir = Path("./train_opus/")
    test_dir = Path('./test_opus/')
    
    train_files = glob(str(train_dir / "audio" / "*.opus"))
    test_files = glob(str(test_dir / "audio" / "*.opus"))
    
    # Загрузка разметки
    wb_path = train_dir / "word_bounds.json"
    with open(wb_path, "r", encoding="utf-8") as f:
        word_bounds = json.load(f)
    
    print(f"Найдено train файлов: {len(train_files)}")
    print(f"Найдено test файлов: {len(test_files)}")
    print(f"Разметка содержит: {len(word_bounds)} записей")
    
    # Создаем списки файлов
    pos_samples = []
    neg_samples = []
    
    for f in tqdm(train_files, desc="Создание списков файлов"):
        fid = Path(f).stem
        if fid in word_bounds:
            bounds_data = word_bounds[fid]
            if isinstance(bounds_data, list) and len(bounds_data) == 2:
                pos_samples.append((Path(f), (float(bounds_data[0]), float(bounds_data[1])), 1))
        else:
            neg_samples.append((Path(f), None, 0))
    
    print(f"Положительные примеры: {len(pos_samples)}")
    print(f"Отрицательные примеры: {len(neg_samples)}")
    
    # Балансировка
    min_samples = min(len(pos_samples), len(neg_samples))
    if len(pos_samples) > min_samples:
        pos_samples = random.sample(pos_samples, min_samples)
    if len(neg_samples) > min_samples:
        neg_samples = random.sample(neg_samples, min_samples)
    
    print(f"После балансировки: {len(pos_samples)} положительных, {len(neg_samples)} отрицательных")
    
    # Разделение на train/val
    pos_train, pos_val = train_test_split(pos_samples, test_size=0.05, random_state=185)
    neg_train, neg_val = train_test_split(neg_samples, test_size=0.05, random_state=185)
    
    train_samples = pos_train + neg_train
    val_samples = pos_val + neg_val
    
    random.shuffle(train_samples)
    random.shuffle(val_samples)
    
    print(f"Train samples: {len(train_samples)}")
    print(f"Val samples: {len(val_samples)}")
    
    # return (pos_train[:1000], neg_train[:1000]), (pos_val[:1000], neg_val[:1000]), test_files
    return (pos_train, neg_train), (pos_val, neg_val), test_files

def create_professional_data_loaders(train_data, val_data):
    """Профессиональные DataLoader'ы для всех данных"""
    pos_train, neg_train = train_data
    pos_val, neg_val = val_data
    
    train_samples = pos_train + neg_train
    val_samples = pos_val + neg_val
    
    random.shuffle(train_samples)
    random.shuffle(val_samples)
    
    print(f"Профессиональные данные:")
    print(f"Train: {len(train_samples)} samples")
    print(f"Val: {len(val_samples)} samples")
    print(f"Segment size: {Config.SEGMENT_SIZE} samples ({Config.SEGMENT_DURATION} sec)")
    
    train_dataset = AdvancedKWSDataset(
        file_list=train_samples,
        seg_size_samples=Config.SEGMENT_SIZE,
        sr=Config.SAMPLE_RATE,
        augment=True
    )
    
    val_dataset = AdvancedKWSDataset(
        file_list=val_samples,
        seg_size_samples=Config.SEGMENT_SIZE,
        sr=Config.SAMPLE_RATE,
        augment=False
    )
    
    # Оптимальный батч для профессионального обучения
    train_loader = DataLoader(
        train_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        num_workers=Config.NUM_WORKERS,
        collate_fn=custom_collate_fn,
        drop_last=True,
        pin_memory=True,
        persistent_workers=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=Config.BATCH_SIZE,
        shuffle=False,
        num_workers=Config.NUM_WORKERS,
        collate_fn=custom_collate_fn,
        pin_memory=True,
        persistent_workers=True
    )
    
    return train_loader, val_loader

In [16]:
# ПРЕДОБУЧЕННАЯ МОДЕЛЬ WAV2VEC2 ДЛЯ KWS
# ИСПРАВЛЕННАЯ ПРЕДОБУЧЕННАЯ МОДЕЛЬ
import torch
import torch.nn as nn
import pytorch_lightning as pl
from transformers import AutoModel, AutoProcessor, Wav2Vec2Processor, Wav2Vec2Model
from torchmetrics import F1Score, Accuracy, Precision, Recall

class PretrainedKWSModel(pl.LightningModule):
    """Исправленная предобученная модель для KWS"""
    
    def __init__(self, num_classes=1, lr=Config.LEARNING_RATE):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        
        # ИСПРАВЛЕНИЕ 1: Используем рабочую модель и правильный способ загрузки
        model_name = Config.model_name
        # model_name = "facebook/wav2vec2-base-960h"  # Эта модель точно работает
        # model_name = 'bond005/wav2vec2-large-ru-golos'
        
        try:
            # Пробуем загрузить модель с помощью AutoModel
            print(f"🔄 Загружаем модель {model_name}...")
            self.wav2vec2 = AutoModel.from_pretrained(model_name)
            print("✅ Модель успешно загружена!")
        except Exception as e:
            print(f"❌ Ошибка загрузки: {e}")
            print("🔄 Пробуем альтернативный способ...")
            # Альтернативный способ загрузки
            self.wav2vec2 = Wav2Vec2Model.from_pretrained(model_name, from_tf=False)
        
        # ИСПРАВЛЕНИЕ 2: Правильная заморозка слоев
        # self.freeze_encoder_layers()
        
        # Классификатор
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.wav2vec2.config.hidden_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes),
        )
        
        self.loss_fn = nn.BCEWithLogitsLoss()
        
        # Метрики
        self.train_f1 = F1Score(task='binary')
        self.val_f1 = F1Score(task='binary')
        self.train_accuracy = Accuracy(task='binary')
        self.val_accuracy = Accuracy(task='binary')
        
        self.best_val_f1 = 0.0
        
    def freeze_encoder_layers(self):
        """Заморозка слоев с информативным выводом"""
        # Замораживаем feature extractor
        for param in self.wav2vec2.feature_extractor.parameters():
            param.requires_grad = False
            
        # Замораживаем первые 8 слоев encoder'а
        for i, layer in enumerate(self.wav2vec2.encoder.layers[:8]):
            for param in layer.parameters():
                param.requires_grad = False
                
        # Считаем обучаемые и замороженные параметры
        total_params = sum(p.numel() for p in self.wav2vec2.parameters())
        trainable_params = sum(p.numel() for p in self.wav2vec2.parameters() if p.requires_grad)
        frozen_params = total_params - trainable_params
        
        print(f"🧊 Заморозка завершена:")
        print(f"   Всего параметров: {total_params:,}")
        print(f"   Заморожено: {frozen_params:,} ({frozen_params/total_params*100:.1f}%)")
        print(f"   Обучаемых: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")
        
    def forward(self, x):
        # Нормализация аудио
        x = (x - x.mean(dim=1, keepdim=True)) / (x.std(dim=1, keepdim=True) + 1e-8)
        
        # Пропускаем через Wav2Vec2
        outputs = self.wav2vec2(x)
        
        # Берем скрытые состояния
        hidden_states = outputs.last_hidden_state
        
        # Глобальное среднее pooling
        pooled_output = hidden_states.mean(dim=1)
        
        # Классификация
        logits = self.classifier(pooled_output)
        return logits.squeeze(-1)
    
    # Остальные методы остаются без изменений
    def training_step(self, batch, batch_idx):
        x, y, aux = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.float())
        
        preds = torch.sigmoid(y_hat)
        self.train_f1(preds, y)
        self.train_accuracy(preds, y)
        
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_f1', self.train_f1, prog_bar=True)
        self.log('train_acc', self.train_accuracy, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y, aux = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y.float())
        
        preds = torch.sigmoid(y_hat)
        self.val_f1(preds, y)
        self.val_accuracy(preds, y)
        
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_f1', self.val_f1, prog_bar=True)
        self.log('val_acc', self.val_accuracy, prog_bar=True)
        
        return loss
    
    # def configure_optimizers(self):
    #     optimizer = torch.optim.AdamW([
    #         {'params': self.wav2vec2.parameters(), 'lr': self.lr * 0.1},
    #         {'params': self.classifier.parameters(), 'lr': self.lr}
    #     ], weight_decay=1e-4)
        
    #     return optimizer

    def configure_optimizers(self):
        # Обучаем все параметры (wav2vec2 и classifier) с одним LR
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=1e-4)
        return optimizer


# Альтернативная легкая предобученная модель (если Wav2Vec2 слишком тяжелая)

def train_pretrained_model(train_loader, val_loader, use_light_model=False):
    """Обучение предобученной модели"""
    print("\n=== ОБУЧЕНИЕ С ПРЕДОБУЧЕННОЙ МОДЕЛЬЮ ===")
    
    # Создаем директорию для чекпоинтов
    Path(Config.CHECKPOINT_DIR).mkdir(exist_ok=True)
    
    if use_light_model:
        model = LightPretrainedKWSModel(lr=Config.LEARNING_RATE)
        print("Используется облегченная предобученная модель")
    else:
        model = PretrainedKWSModel(lr=Config.LEARNING_RATE)
        print("Используется Wav2Vec2 предобученная модель")
    
    checkpoint_callback = ModelCheckpoint(
        monitor='val_f1',
        dirpath=Config.CHECKPOINT_DIR,
        filename='kws-pretrained-{epoch:02d}-{val_f1:.3f}',
        save_top_k=3,
        mode='max',
        save_last=True
    )
    
    early_stop_callback = EarlyStopping(
        monitor='val_f1',
        patience=10,
        mode='max',
        min_delta=0.001
    )
    
    lr_monitor = LearningRateMonitor(logging_interval='step')
    
    trainer = pl.Trainer(
        max_epochs=3,
        callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
        accelerator='gpu',
        devices=1,
        precision='16-mixed',
        accumulate_grad_batches=1,
        log_every_n_steps=20,
        check_val_every_n_epoch=1,
        gradient_clip_val=1.0,
        benchmark=True,
        deterministic=False,
        enable_progress_bar=True,
    )
    
    print("Конфигурация предобученной модели:")
    print(f"Параметров: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Обучаемых параметров: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    print(f"Размер сегмента: {Config.SEGMENT_SIZE} samples ({Config.SEGMENT_DURATION} сек)")
    print(f"Batch size: {Config.BATCH_SIZE}")
    print(f"Learning rate: {Config.LEARNING_RATE}")
    print(f"Train samples: {len(train_loader.dataset)}")
    print(f"Val samples: {len(val_loader.dataset)}")
    
    print("Начинаем обучение с предобученной моделью...")
    trainer.fit(model, train_loader, val_loader)
    
    return model, trainer

def load_best_model(use_light_model=False):
    """Загрузка лучшей сохраненной модели"""
    if Path(Config.BEST_MODEL_PATH).exists():
        checkpoint = torch.load(Config.BEST_MODEL_PATH)
        model = PretrainedKWSModel()
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✅ Загружена лучшая модель (F1: {checkpoint['val_f1']:.4f})")
        return model
    else:
        print("❌ Лучшая модель не найдена")
        return None

In [6]:
# ШАГ 1: Загружаем данные
print("ШАГ 1: Загрузка данных...")
train_data, val_data, test_files = load_and_prepare_data()

# ШАГ 2: Создаем DataLoader'ы с аугментациями
print("ШАГ 2: Создание DataLoader'ов с аугментациями...")
pro_train_loader, pro_val_loader = create_professional_data_loaders(train_data, val_data)

# ШАГ 3: Обучаем предобученную модель
print("ШАГ 3: Обучение предобученной модели...")

# Выберите одну из моделей:
# use_light_model=True для легкой версии (быстрее, меньше памяти)
# use_light_model=False для Wav2Vec2 (лучшее качество, но требует больше ресурсов)

model, trainer = train_pretrained_model(pro_train_loader, pro_val_loader, use_light_model=False)

ШАГ 1: Загрузка данных...
=== ЗАГРУЗКА ДАННЫХ ===
Конфигурация: сегмент 2.0 сек (32000 samples)
Найдено train файлов: 90000
Найдено test файлов: 27000
Разметка содержит: 45000 записей


Создание списков файлов: 100%|██████████| 90000/90000 [00:02<00:00, 34912.45it/s]


Положительные примеры: 45000
Отрицательные примеры: 45000
После балансировки: 45000 положительных, 45000 отрицательных
Train samples: 85500
Val samples: 4500
ШАГ 2: Создание DataLoader'ов с аугментациями...
Профессиональные данные:
Train: 85500 samples
Val: 4500 samples
Segment size: 32000 samples (2.0 sec)
ШАГ 3: Обучение предобученной модели...

=== ОБУЧЕНИЕ С ПРЕДОБУЧЕННОЙ МОДЕЛЬЮ ===
🔄 Загружаем модель UrukHan/wav2vec2-russian...


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/user/.local/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/user/Desktop/a/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


✅ Модель успешно загружена!
Используется Wav2Vec2 предобученная модель
Конфигурация предобученной модели:
Параметров: 316,096,129
Обучаемых параметров: 316,096,129
Размер сегмента: 32000 samples (2.0 сек)
Batch size: 32
Learning rate: 0.0001
Train samples: 85500
Val samples: 4500
Начинаем обучение с предобученной моделью...


...

Validation: |          | 0/? [00:00<?, ?it/s]

/home/user/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 20. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [2]:
# val_f1 0.797
# val_f1 0.894 UrukHan/wav2vec2-russian

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.auto import tqdm
import json
from pathlib import Path
from glob import glob
import pandas as pd
import torchaudio
import librosa
import pytorch_lightning as pl
from transformers import AutoModel

In [18]:
from pathlib import Path

# Импортируем модель (предполагаем, что класс PretrainedKWSModel уже определён)
ckpt_path = "checkpoints/kws-pretrained-epoch=02-val_f1=0.979.ckpt"

# Загружаем модель
model = PretrainedKWSModel.load_from_checkpoint(ckpt_path)

# Переносим на нужное устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print(f"✅ Модель успешно загружена из {ckpt_path}")
print(f"   Устройство: {device}")

🔄 Загружаем модель UrukHan/wav2vec2-russian...
✅ Модель успешно загружена!
✅ Модель успешно загружена из checkpoints/kws-pretrained-epoch=02-val_f1=0.979.ckpt
   Устройство: cuda


In [19]:
import concurrent.futures
from torch.utils.data import Dataset, DataLoader

class KWSDataset(Dataset):
    """
    Dataset для KWS-инференса: режет файлы на окна и возвращает (аудио, file_id)
    """
    def __init__(self, file_paths, sample_rate=16000, window_size=16000, stride=4000, max_duration=10.0, preprocess_fn=None, load_fn=None):
        self.file_paths = file_paths
        self.sample_rate = sample_rate
        self.window_size = window_size
        self.stride = stride
        self.max_samples = int(max_duration * sample_rate)
        self.preprocess_fn = preprocess_fn
        self.load_fn = load_fn

        # Предварительная загрузка в потоках
        self.index_map = []
        self.windows = []
        self.file_ids = []

        print("🧠 Подготовка датасета инференса...")

        def process_file(file_path):
            try:
                audio = self.load_fn(file_path)
                if audio is None:
                    return []
                T = len(audio)
                if T > self.max_samples:
                    audio = audio[:self.max_samples]
                    T = self.max_samples
                audio = self.preprocess_fn(audio)
                windows = []
                if T < window_size:
                    padded = torch.zeros(window_size)
                    padded[:T] = audio
                    windows.append(padded)
                else:
                    for start in range(0, T - window_size + 1, stride):
                        end = start + window_size
                        windows.append(audio[start:end])
                if not windows:
                    windows = [torch.zeros(window_size)]
                return [(Path(file_path).stem, w) for w in windows]
            except Exception as e:
                print(f"❌ Ошибка загрузки {file_path}: {e}")
                return []

        # Многопоточная подготовка
        with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
            for result in tqdm(executor.map(process_file, file_paths), total=len(file_paths), desc="🎧 Подготовка окон", colour='blue'):
                for file_id, w in result:
                    self.file_ids.append(file_id)
                    self.windows.append(w)
                    self.index_map.append(file_id)

        print(f"✅ Всего окон: {len(self.windows)} для {len(file_paths)} файлов")

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        return self.windows[idx], self.index_map[idx]


In [20]:
# ИНФЕРЕНС КЛАСС ДЛЯ WAV2VEC2
class Wav2Vec2KWSInference:
    """Инференс для Wav2Vec2 модели KWS"""
    
    def __init__(self, model_path=None, model=None, device='auto'):
        """
        Инициализация инференса для Wav2Vec2 модели
        """
        if device == 'auto':
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(device)
            
        print(f"🚀 Инициализация Wav2Vec2 инференса на устройстве: {self.device}")
        
        # Загружаем модель
        if model is not None:
            self.model = model
        else:
            raise ValueError("Необходимо указать model_path или передать готовую модель")
            
        self.model.to(self.device)
        self.model.eval()
        
        # Конфигурация (должна совпадать с обучением)
        self.sample_rate = 16000
        self.segment_duration = Config.SEGMENT_DURATION
        self.segment_size = int(self.sample_rate * self.segment_duration)
    
    def dataloader_predict(self, test_files, threshold=0.5, batch_size=128, num_workers=4,
                       stride=4000, max_duration=10.0):
        """
        Новый инференс на основе DataLoader, повторяющий логику sliding_window_predict.
        """
        print(f"🚀 Запуск инференса через DataLoader ({len(test_files)} файлов)...")

        dataset = KWSDataset(
            file_paths=test_files,
            sample_rate=self.sample_rate,
            window_size=self.segment_size,
            stride=stride,
            max_duration=max_duration,
            preprocess_fn=self.preprocess_audio,
            load_fn=self.load_audio_16k,
        )

        loader = DataLoader(
            dataset,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=True,
            shuffle=False,
        )

        self.model.eval()
        self.model.to(self.device)

        all_probs = []
        all_ids = []

        with torch.no_grad(), torch.cuda.amp.autocast():
            for batch, file_ids in tqdm(loader, desc="🚀 Инференс батчами", colour='green'):
                batch = batch.to(self.device)
                preds = self.model(batch)
                probs = torch.sigmoid(preds).detach().cpu().numpy().flatten()
                all_probs.extend(probs)
                all_ids.extend(file_ids)

        # Агрегация по файлам
        df = pd.DataFrame({"id": all_ids, "prob": all_probs})
        grouped = df.groupby("id")["prob"].agg(["max", "count"]).reset_index()
        grouped["label"] = (grouped["max"] >= threshold).astype(int)
        grouped.rename(columns={"max": "max_prob", "count": "window_count"}, inplace=True)

        print("✅ Инференс завершён!")
        return (
            grouped["id"].tolist(),
            grouped["label"].tolist(),
            grouped["max_prob"].tolist(),
            grouped["window_count"].tolist(),
        )

                
    def load_audio_16k(self, path, sr_target=16000):
        """Загрузка аудио файла с ресемплированием до 16kHz"""
        path = str(path)
        try:
            wav, sr = torchaudio.load(path)
            if wav.dim() == 2:
                if wav.size(0) > 1:
                    wav = wav.mean(dim=0, keepdim=True)
                wav = wav.squeeze(0)

            if sr != sr_target:
                wav = torchaudio.functional.resample(wav, orig_freq=sr, new_freq=sr_target)

            if wav.dtype != torch.float32:
                wav = wav.float()
            return wav
        except Exception as e:
            try:
                y, sr = librosa.load(path, sr=sr_target, mono=True)
                wav = torch.from_numpy(y.astype(np.float32))
                return wav
            except Exception as e2:
                print(f"❌ Ошибка загрузки аудио {path}: {e2}")
                return torch.zeros(sr_target, dtype=torch.float32)
    
    def preprocess_audio(self, audio):
        """Предобработка аудио (нормализация)"""
        # Нормализация по пику
        peak = audio.abs().max().item()
        if peak > 1e-6:
            audio = audio / peak
            
        return audio
    
    def ensure_length(self, wav, target_len):
        """Приведение аудио к нужной длине"""
        T = wav.numel()
        if T == target_len:
            return wav
        if T > target_len:
            return wav[:target_len]
        pad = target_len - T
        return F.pad(wav, (0, pad))
    
    def sliding_window_predict(self, test_files, window_size=None, stride=4000,
                           threshold=0.5, batch_size=64, max_duration=10.0):
        """
        Предсказание с использованием скользящего окна, батчами и на GPU.
        """
        if window_size is None:
            window_size = self.segment_size

        self.model.eval()

        print(f"🔍 Wav2Vec2 батч-инференс: {len(test_files)} файлов")
        print(f"   Окно: {window_size} samples ({window_size/self.sample_rate:.1f} сек)")
        print(f"   Шаг: {stride} samples ({stride/self.sample_rate:.2f} сек)")
        print(f"   Batch size: {batch_size}, threshold: {threshold}")

        # === 1. Подготовка окон для всех файлов ===
        all_file_ids = []
        all_windows = []
        file_window_map = []

        for file_path in tqdm(test_files, desc="🎧 Подготовка аудио", colour='blue'):
            filename = Path(file_path).stem
            try:
                audio = self.load_audio_16k(file_path)
                T = len(audio)

                # Ограничение длины
                max_samples = int(max_duration * self.sample_rate)
                if T > max_samples:
                    audio = audio[:max_samples]
                    T = max_samples

                audio = self.preprocess_audio(audio)

                # Создаём окна
                windows = []
                if T < window_size:
                    padded = torch.zeros(window_size)
                    padded[:T] = audio
                    windows.append(padded)
                else:
                    for start in range(0, T - window_size + 1, stride):
                        end = start + window_size
                        windows.append(audio[start:end])

                if not windows:
                    windows = [torch.zeros(window_size)]

                # Сохраняем все окна и соответствие
                start_idx = len(all_windows)
                all_windows.extend(windows)
                end_idx = len(all_windows)
                file_window_map.append((filename, start_idx, end_idx))
                all_file_ids.append(filename)

            except Exception as e:
                print(f"❌ Ошибка при подготовке {file_path}: {e}")
                file_window_map.append((filename, None, None))
                all_file_ids.append(filename)

        if not all_windows:
            raise ValueError("Нет окон для обработки!")

        all_windows = torch.stack(all_windows)
        print(f"📦 Всего окон: {len(all_windows)}")

        # === 2. Батч-инференс на GPU ===
        all_probs = np.zeros(len(all_windows), dtype=np.float32)
        self.model.to(self.device)

        with torch.no_grad():
            for i in tqdm(range(0, len(all_windows), batch_size), desc="🚀 Инференс", colour='green'):
                batch = all_windows[i:i + batch_size].to(self.device)
                preds = self.model(batch)
                probs = torch.sigmoid(preds).detach().cpu().numpy().flatten()
                all_probs[i:i + len(probs)] = probs

        # === 3. Агрегация по файлам ===
        all_preds = []
        all_max_probs = []
        all_window_counts = []

        for filename, start, end in file_window_map:
            if start is None:
                all_preds.append(0)
                all_max_probs.append(0.0)
                all_window_counts.append(0)
                continue

            probs = all_probs[start:end]
            max_prob = float(np.max(probs)) if len(probs) else 0.0
            label = 1 if max_prob >= threshold else 0

            all_preds.append(label)
            all_max_probs.append(max_prob)
            all_window_counts.append(end - start)

        print("✅ Батч-инференс завершён!")

        return all_file_ids, all_preds, all_max_probs, all_window_counts

    
    def predict_single_file(self, audio_path, threshold=0.5):
        """Предсказание для одного файла"""
        results = self.sliding_window_predict(
            [audio_path], 
            threshold=threshold,
            batch_size=1
        )
        return {
            'id': results[0][0],
            'prediction': results[1][0],
            'confidence': results[2][0],
            'windows_processed': results[3][0]
        }
    
    def run_predictions(self, test_dir, output_dir="./output", 
                       threshold=0.5, batch_size=64, **kwargs):
        """Запуск предсказаний для всех файлов в директории"""
        print("=== 🎯 ЗАПУСК WAV2VEC2 ИНФЕРЕНСА ===")
        
        # Создаем выходную директорию
        Path(output_dir).mkdir(exist_ok=True)
        
        # Находим тестовые файлы
        test_files = glob(str(Path(test_dir) / "audio" / "*.opus"))
        if not test_files:
            # Пробуем другие расширения
            test_files = glob(str(Path(test_dir) / "audio" / "*.wav"))
            test_files += glob(str(Path(test_dir) / "audio" / "*.mp3"))
            
        print(f"📁 Найдено тестовых файлов: {len(test_files)}")
        
        if not test_files:
            raise ValueError("Не найдено аудио файлов для обработки")
        
        # Основная стратегия - максимум по окнам
        all_ids, all_preds, all_probs, window_counts = self.dataloader_predict(
            test_files=test_files,
            threshold=threshold,
            batch_size=batch_size,
        )

        
        # Создаем DataFrame для submission
        subm = pd.DataFrame({
            "id": all_ids,
            "label": all_preds
        })
        
        # Дополнительная аналитика
        analytics_df = pd.DataFrame({
            "id": all_ids,
            "label": all_preds,
            "max_prob": all_probs,
            "window_count": window_counts
        })
        
        # Сохраняем основной submission
        subm_path = Path(output_dir) / "submission.csv"
        subm.to_csv(subm_path, index=False)
        
        # Сохраняем аналитику
        analytics_path = Path(output_dir) / "prediction_analytics.csv"
        analytics_df.to_csv(analytics_path, index=False)
        
        # Статистика
        positive_files = sum(all_preds)
        total_files = len(all_preds)
        
        print(f"\n📊 РЕЗУЛЬТАТЫ WAV2VEC2 ИНФЕРЕНСА:")
        print(f"   Всего файлов: {total_files}")
        print(f"   Обнаружено ключевых слов: {positive_files}")
        print(f"   Доля положительных: {positive_files/total_files:.3f}")
        print(f"   Средняя уверенность: {np.mean(all_probs):.3f}")
        print(f"   Максимальная уверенность: {np.max(all_probs):.3f}")
        print(f"   Минимальная уверенность: {np.min(all_probs):.3f}")
        print(f"   Среднее число окон на файл: {np.mean(window_counts):.1f}")
        print(f"\n💾 Файлы сохранены:")
        print(f"   Submission: {subm_path}")
        print(f"   Аналитика: {analytics_path}")
        
        # Показываем примеры с высокой уверенностью
        if positive_files > 0:
            print(f"\n🎯 Топ-5 файлов с высокой уверенностью:")
            high_confidence = analytics_df.nlargest(5, 'max_prob')[['id', 'max_prob', 'label']]
            print(high_confidence.to_string(index=False))
        else:
            print(f"\n⚠️  Положительных файлов не обнаружено")
        
        return subm, analytics_df

# УПРОЩЕННАЯ ВЕРСИЯ ДЛЯ БЫСТРОГО ИСПОЛЬЗОВАНИЯ
def create_inference_from_trained_model(model, device='auto'):
    """Создание инференса из уже обученной модели"""
    return Wav2Vec2KWSInference(model=model, device=device)

In [23]:
# Вариант 1: Загрузка из файла
# inference = Wav2Vec2KWSInference(
#     model_path=model,  # путь к вашей обученной модели
#     device='auto'
# )

# Вариант 2: Использование уже загруженной модели
inference = create_inference_from_trained_model(model)

# Запускаем инференс
submission, analytics = inference.run_predictions(
    test_dir='./test_opus/',
    threshold=0.5,  # Можно настроить порог
    batch_size=512,   # Меньше batch size для Wav2Vec2
)

print("\n✅ Wav2Vec2 инференс завершен!")

🚀 Инициализация Wav2Vec2 инференса на устройстве: cuda
=== 🎯 ЗАПУСК WAV2VEC2 ИНФЕРЕНСА ===
📁 Найдено тестовых файлов: 27000
🚀 Запуск инференса через DataLoader (27000 файлов)...
🧠 Подготовка датасета инференса...


🎧 Подготовка окон:   0%|          | 0/27000 [00:00<?, ?it/s]

✅ Всего окон: 243000 для 27000 файлов


/tmp/ipykernel_10154/182756481.py:61: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


🚀 Инференс батчами:   0%|          | 0/475 [00:00<?, ?it/s]

✅ Инференс завершён!

📊 РЕЗУЛЬТАТЫ WAV2VEC2 ИНФЕРЕНСА:
   Всего файлов: 27000
   Обнаружено ключевых слов: 14449
   Доля положительных: 0.535
   Средняя уверенность: 0.535
   Максимальная уверенность: 1.000
   Минимальная уверенность: 0.000
   Среднее число окон на файл: 9.0

💾 Файлы сохранены:
   Submission: output/submission.csv
   Аналитика: output/prediction_analytics.csv

🎯 Топ-5 файлов с высокой уверенностью:
                                      id  max_prob  label
7859597975706849028526902407132641044648  1.000000      1
0021302091384145928809761101311661532340  0.999512      1
0023081956559018297738505849805505970846  0.999512      1
0039469348250598917294113941175387061903  0.999512      1
0041440161072449292443056777444708933051  0.999512      1

✅ Wav2Vec2 инференс завершен!


In [84]:
analytics['label'] = (analytics['max_prob'] > 0.85).astype(int)

In [85]:
analytics['label'].value_counts()

1    14014
0    12986
Name: label, dtype: int64

In [86]:
analytics[['id', 'label']].to_csv('sub_wav2vec2_1_85.csv', index=False)